# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
infile = os.path.join("Output", "city_weather_data.csv")

print(infile)
city_data = pd.read_csv(infile)
city_data.dtypes


Output\city_weather_data.csv


City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
gmaps.configure(api_key=g_key)

# Create a variable for lat, long
city_locations = city_data[["Lat", "Lng"]]

# Create a variable for humidity'
city_humidity = city_data["Humidity"]

# Create the  figure
figure_layout = {
    'width': '800px',
    'height': '580px',
    'border': '3px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(zoom_level = 2, center = (24.288292, 11.701573), layout=figure_layout)

# Create the heatmap layer for city locations and humidity
heat_layer = gmaps.heatmap_layer(city_locations, weights=city_humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

# Display the figure
fig

Figure(layout=FigureLayout(border='3px solid black', height='580px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
city_data_filtered = city_data.loc[(city_data['Max Temp'].between(70, 80)) & (city_data['Wind Speed'] < 10) & (city_data['Cloudiness'] < 5)].copy()  
city_data_clean = city_data_filtered.dropna()
city_data_clean

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,Victoria,-32.62,-60.15,73.00,34,0,4.00,AR,1600032965
106,Yagoua,10.34,15.23,72.70,88,3,5.41,CM,1600032993
220,Sartana,47.17,37.70,71.01,100,0,5.01,UA,1600033025
224,Al Bardīyah,31.76,25.09,74.25,75,0,4.94,LY,1600033026
237,Kankakee,41.15,-87.87,77.00,53,1,7.63,US,1600033030
246,Steamboat Springs,40.48,-106.83,79.00,13,1,5.82,US,1600032861
260,Vignola,44.48,11.00,75.00,73,0,3.02,IT,1600033035
267,Marsá Maţrūḩ,31.35,27.25,77.00,73,0,5.82,EG,1600033037
285,Half Moon Bay,37.46,-122.43,71.01,77,1,8.05,US,1600032822
295,Mount Isa,-20.73,139.50,71.60,35,0,8.05,AU,1600032972


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = city_data_clean
hotel_df = hotel_df.reset_index(drop=True)

In [18]:
hotel_df['Hotel Name'] = ""
hotel_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Create a dictionary variable to store the parameters
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key,
}

for index, row in hotel_df.iterrows():

    # get restaurant type from DataFrame
    lat_longs = f"{row['Lat']},{row['Lng']}"

    # add a new key to our params dictionary that will store our keyword to search for
    params['location'] = lat_longs

    # assemble url and make API request
#    print(f"Retrieving Results for Index {index}: {row['City']} {lat_longs}.")
    
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        print(f"The closest hotel in {row['City']} is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print(f"No hotel found in {row['City']}")

hotel_df

The closest hotel in Victoria is Río Manso apart.
The closest hotel in Yagoua is HÔTEL LE PROGRÈS.
The closest hotel in Sartana is Сартана.
The closest hotel in Al Bardīyah is بورتو بردية.
The closest hotel in Kankakee is Holiday Inn Express & Suites Bourbonnais (Kankakee/Bradley).
The closest hotel in Steamboat Springs is Steamboat Grand.
The closest hotel in Vignola is Hotel La Cartiera S.
The closest hotel in Marsá Maţrūḩ is Beau Site.
The closest hotel in Half Moon Bay is The Ritz-Carlton, Half Moon Bay.
The closest hotel in Mount Isa is ibis Styles Mt Isa Verona.
No hotel found in Winona
The closest hotel in San Benedetto del Tronto is Hotel Imperial.
The closest hotel in Angoulême is Mercure Angoulême - Hôtel de France.
The closest hotel in Dalbandin is Police station anti NARCOTICS FORCE.
The closest hotel in Mardin is Reyhani Kasri Hotel.
The closest hotel in Prata is Hotel Pontal Aluga-se Quitinetes e suítes mensalmente.
The closest hotel in Biograd na Moru is Hotel Palma.
The

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Victoria,-32.62,-60.15,73.00,34,0,4.00,AR,1600032965,Río Manso apart
1,Yagoua,10.34,15.23,72.70,88,3,5.41,CM,1600032993,HÔTEL LE PROGRÈS
2,Sartana,47.17,37.70,71.01,100,0,5.01,UA,1600033025,Сартана
3,Al Bardīyah,31.76,25.09,74.25,75,0,4.94,LY,1600033026,بورتو بردية
4,Kankakee,41.15,-87.87,77.00,53,1,7.63,US,1600033030,Holiday Inn Express & Suites Bourbonnais (Kank...
5,Steamboat Springs,40.48,-106.83,79.00,13,1,5.82,US,1600032861,Steamboat Grand
6,Vignola,44.48,11.00,75.00,73,0,3.02,IT,1600033035,Hotel La Cartiera S
7,Marsá Maţrūḩ,31.35,27.25,77.00,73,0,5.82,EG,1600033037,Beau Site
8,Half Moon Bay,37.46,-122.43,71.01,77,1,8.05,US,1600032822,"The Ritz-Carlton, Half Moon Bay"
9,Mount Isa,-20.73,139.50,71.60,35,0,8.05,AU,1600032972,ibis Styles Mt Isa Verona


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
state_layer =   
# Add the layer to the figure
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='3px solid black', height='580px', margin='0 auto 0 auto', padding='1px', wi…